# 3. Hyperparameter Tuning Experiments

**Student:** Souhaib Othmani

## Purpose
- Select one hyperparameter to tune (learning rate, batch size, etc.)
- Run 3 experiments with well-chosen values
- Compare performance across configurations
- Identify best-performing variant
- Analyze effects of hyperparameter changes

In [ ]:
# TODO: Import libraries and load setup from previous notebooks

In [ ]:
# TODO: Document hyperparameter selection
# - Explain why this parameter was chosen
# - Define 3 well-chosen values (logarithmic spacing if continuous)
# - Example: learning rates [0.0001, 0.001, 0.01]

In [ ]:
# TODO: Experiment 1 - Train with first hyperparameter value
# - Load baseline model
# - Modify only the selected hyperparameter
# - Keep all other settings identical to baseline
# - Log to TensorBoard with distinct run name
# - Save checkpoint

In [ ]:
# TODO: Experiment 2 - Train with second hyperparameter value

In [ ]:
# TODO: Experiment 3 - Train with third hyperparameter value

In [ ]:
# TODO: Compare all 3 configurations
# - Plot training curves for all 3 on same graph
# - Compare final metrics (accuracy, loss)
# - Analyze convergence behavior

In [ ]:
# TODO: Select best configuration
# - Justify selection based on metrics
# - Note which configuration performed best

In [ ]:
# TODO: Write tuning analysis (1-2 paragraphs)
# - Discuss effects of hyperparameter changes
# - Explain why certain values work better
# - Note impact on training dynamics

In [ ]:
# TODO: Save tuned model checkpoints
# - Location: saved_models/tuned_variant_[param]_[value]/